In [293]:
import json
from haralyzer import HarParser, HarPage
from bs4 import BeautifulSoup
import pandas as pd

filename = input('Enter the name of the file: ')
with open(f'College Schedule Downloader/HAR Files/{filename}.har', 'r') as f: 
    har_parser = HarParser(json.loads(f.read()))
data = har_parser.har_data

In [294]:
classDF = pd.DataFrame(columns=['Class Name','Unique', 'Day', 'Hour', 'Room', 'Instruction Mode', 'Instructor','Status', 'Flags', 'Core'])

for i in range (len(data['entries'])):
    html_content = data['entries'][i]['response']['content']['text']
    soup = BeautifulSoup(html_content, 'html.parser')
    tables = soup.find_all('table')
    for table in tables:
        class_name = ''
        # Find all the rows in the current table
        rows = table.find_all('tr')
        
        # Iterate through the rows
        for i,row in enumerate(rows):
            # Find all the cells in the current row
            # print(row, '\n')
            cells = row.find_all(['th','td'])

            
            # Process and print cell data
            header = False
            rowList = []
            for cell in cells:
                if 'course_header' in cell.get('class', []):
                    class_name = cell.get_text()
                    
                    header = True
                else:
                    key = cell.get('data-th', '')
                    # print(key)
                    value = cell.get_text(strip=True)
                    rowList.append(value)

                    # if we detect a header, we don't want to print a row
            if header == False and i>0:
                # print(class_name)
                rowList.insert(0, class_name)
                classDF.loc[len(classDF)] =  rowList
                rowList = []

                

                # check the ending if cropping data!!!
                
                # classDF.loc[len(classDF)] =  classifications

            
            
classDF.drop_duplicates(inplace=True)


In [295]:
classDF

,Class Name,Unique,Day,Hour,Room,Instruction Mode,Instructor,Status,Flags,Core
0,HDO 382 APPROACHES TO LEADERSHIP,29610,S,8:30 a.m.-11:30 a.m.,IC2 3.128,Face-to-face,"SURI, JEREMI",open,,
1,ELP 384D ADMIN OF INDIVIDUAL SCHOOL,11210,S,9:00 a.m.-12:00 p.m.,SZB 2.418,Hybrid or Blended,"CANTU, DEBRA",open; reserved,,
2,HDO 391 QUANTITATIVE RESEARCH METHODS,29625,S,12:30 p.m.-3:30 p.m.,IC2 3.128,Face-to-face,"ROSE, MARY",open,,
3,"ELP 384R SCHL, FAM, COMM ENGAGEMENT",11240,S,1:00 p.m.-4:00 p.m.,SZB 2.418,Hybrid or Blended,"DEMATTHEWS, DAVID EDWARD",open; reserved,,
4,HDO 289 WRITING/RESEARCHING ORGS,29620,S,3:30 p.m.-5:30 p.m.,IC2 3.128,Face-to-face,"BOYLE, CASEY A",open,,


In [296]:
# go through each <tr>. Assign the first course-header class to the current course name
# check if the next <tr> is a course-header class. If so, assign it to the current course name
# if not, then convert the <tr> to a row in the table, and add the course name to that row
classDF.to_csv(f'College Schedule Downloader/CSVs{filename}.csv', index=False)

In [7]:
# go through every csv in the csvs folder, adn combine them into one csv
import glob
import pandas as pd
df = pd.concat(map(pd.read_csv, glob.glob('csvs/*.csv'))).drop_duplicates()
df.to_csv('Ut Course Catalog.csv', index=False)